In [1]:
# Get raw data
with open('input/24.txt', 'r') as f:
    rawinput = f.read().strip()

In [2]:
# Part 1
class Group(object):
    attack_code = {'bludgeoning': 0, 'cold': 1, 'fire': 2, 'radiation': 3, 'slashing': 4}
    def __init__(self, army, desc, boost=0):
        self.army = army
        self.set_stats(desc, boost)
        self.target = None
        
    def __repr__(self):
        return (f'Army: {self.army}, Units: {self.units}, Hit Pts: {self.hit_pts}, '
                +f'Initiative: {self.initiative}, Attack: {self.attack}, Defense: {self.defense}')
        
    def set_stats(self, desc, boost):
        self.units = int((z:=desc.split())[0])
        self.hit_pts = int(z[4])
        self.initiative = int(z[-1])
        self.attack = [(int(z[-6])+boost)*(i==self.attack_code[z[-5]]) 
                       for i in range(5)]
        def_raw = [i for i in ' '.join(z[7:-11])[1:-1].split(';') if i]
        self.defense = [1]*5
        for i in def_raw:
            mult = 2*((z:=i.split())[0]=='weak')
            for j in z[2:]:
                self.defense[self.attack_code[j.strip(',')]] = mult
                
    def eff_power(self):
        return self.units * max(self.attack)
    
def do_battle(groups):
    while True:
        targ_order = [*zip(*sorted([[-j.eff_power(), -j.initiative, i] 
                                    for i,j in enumerate(groups)
                                    if j.units]))][2]
        targ_avail = {i for i,j in enumerate(groups) if j.units}
        for g in targ_order:
            if (z:=sorted([[-sum([groups[g].units*i*j 
                                  for i,j in zip(groups[g].attack, 
                                                 groups[t].defense)]),
                            -groups[t].eff_power(),
                            -groups[t].initiative,
                            t]
                           for t in targ_avail-{g}
                           if groups[t].army != groups[g].army])) and z[0][0]<0:
                groups[g].target = z[0][3]
                targ_avail -= {z[0][3]}
            else:
                groups[g].target = None
    
        attack_order = [*zip(*sorted([[-j.initiative, i] 
                                      for i,j in enumerate(groups)
                                      if j.units]))][1]
        no_change=True
        for g in attack_order:
            if groups[g].units and (t:=groups[g].target) is not None:
                units_lost = min(groups[t].units,
                                 sum([groups[g].units*i*j 
                                      for i,j in zip(groups[g].attack, groups[t].defense)]) 
                                    // groups[t].hit_pts)
                if units_lost:
                    no_change = False
                    groups[t].units -= units_lost
                    
        units_left = [[i,k]
                      for i in 'GB'
                      if (k:=sum([j.units 
                                  for j in groups 
                                  if j.army == i and j.units])) > 0]
        if no_change or len(units_left)==1:
            return units_left
    
groups = [Group('GB'[i], k) 
          for i,j in enumerate(rawinput.split('\n\n'))
          for k in j.split('\n')[1:]]
z[0][1] if len(z:=do_battle(groups))==1 else 'No winner'

23385

In [3]:
# Part 2
def testboost(boost):
    groups = [Group((army:='GB'[i]), k, boost*(army=='G'))
            for i,j in enumerate(rawinput.split('\n\n'))
            for k in j.split('\n')[1:]]
    return do_battle(groups)

i = 0
while len(z:=testboost(10**i)) > 1 or z[0][0] == 'B':
    i += 1
boost_limits = [10**(i-1), 10**i]
while boost_limits[1]-boost_limits[0] > 1:
    boost_limits[int(len(z:=testboost(y))==1 and z[0][0]=='G')] = (y:=sum(boost_limits)//2)

testboost(boost_limits[1])[0][1]

2344